# 7 Solution Methods to Solve the Growth Model with Python

This notebook is part of a computational appendix that accompanies the paper

> MATLAB, Python, Julia: What to Choose in Economics?
> > Coleman, Lyon, Maliar, and Maliar (2017)

In order to run the codes in this notebook you will need to install and configure a few Python packages. We recommend downloading [Anaconda](https://www.continuum.io/downloads) and/or following the instructions on [quantecon.org](https://lectures.quantecon.org/py/getting_started.html). Once your Python installation is up and running, there are a few additional packages you will need in order to run the code here. To do this, you should remove the `# ` in front of the following commands and run the block below.

In [2960]:
# ! pip install git+https://github.com/EconForge/interpolation.py.git
# ! pip install quantecon
# ! pip install csaps

In [2961]:
import numpy as np
import scipy.linalg as la
import scipy.optimize as opt
import time
import quantecon as qe
import matplotlib.pyplot as plt

from collections import namedtuple
from scipy.interpolate import interp1d, UnivariateSpline
from scipy.misc import derivative
from interpolation.complete_poly import (CompletePolynomial,
                                         n_complete, complete_polynomial,
                                         complete_polynomial_der,
                                         _complete_poly_impl,
                                         _complete_poly_impl_vec,
                                         _complete_poly_der_impl,
                                         _complete_poly_der_impl_vec)
from numba import jit, vectorize

## Model

This section gives a short description of the commonly used stochastic Neoclassical growth model.

There is a single infinitely-lived representative agent who consumes and saves using capital. The consumer discounts the future with factor $\beta$ and derives utility from only consumption. Additionally, saved capital will depreciate at $\delta$.

The consumer has access to a Cobb-Douglas technology which uses capital saved from the previous period to produce and is subject to stochastic productivity shocks.

Productivity shocks follow an AR(1) in logs.

The agent's problem can be written recursively using the following Bellman equation


$$
\begin{align}
  V(k_t, z_t) &= \max_{k_{t+1}} u(c_t) + \beta E \left[ V(k_{t+1}, z_{t+1}) \right] \\
  &\text{subject to } \\
  c_t &= z_t f(k_t) + (1 - \delta) k_t - k_{t+1} \\
  \log z_{t+1} &= \rho \log z_t + \sigma \varepsilon
\end{align}
$$

$$
\begin{align*}
    V(X, z) &= \max_{\tilde{c},\tilde{s}} \biggl\{0,  u(c) + \beta E \bigl[V(X', z') \bigr] \biggr\}\\
    & \text{ s.t. } \tilde{c} \geq 0; \qquad \tilde{s} \geq 0 \\
    & \phantom{\text{ s.t. } }N\tilde{c} + N\tilde{s} \leq X \\
    & \phantom{\text{ s.t. } } X' = \max \biggl\{0, \bigl[X - N\tilde{c}  - N\tilde{s} - z \bigl(\frac{\tilde{c}}{\tilde{s}}\bigr)^\phi \bigr](1 + g_x) \biggr\} \\
    & \phantom{\text{ s.t. } } u(\tilde{c}) = \bar{u} + \ln(\tilde{c}) \\
    & \phantom{\text{ s.t. } } z \sim Exp( \bar{z} ) 
\end{align*} 
$$

## Python Code

We begin by defining a `namedtuple` that contains the parameters of our model. This is useful to pass the parameters around to functions that are just-in-time (JIT) compiled by `numba`.

In [2962]:
#
# Create a named tuple type that we can pass into the jitted functions
# so that we don't have to pass parameters one by one
#
Params = namedtuple("Params", ["N", "beta", "g", "a_bar", "eta", "u_bar"])

@jit(nopython=True)
def param_unpack(params):
    "Unpack parameters from the Params type"
    out = (params.N, params.beta, params.g,
           params.a_bar, params.eta, params.u_bar)

    return out

We will then define various helper functions to ensure that we [don't repeat ourselves](https://lectures.quantecon.org/py/writing_good_code.html#don-t-repeat-yourself) and that the inner functions can be JIT compiled.

In [2963]:
#
# Helper functions to make sure things are jitted
#
@vectorize(nopython=True)
def u(c, eta, u_bar):
    "log utility function"
    return -1e10 if c < 1e-10 else (u_bar + np.log(c)) if eta==1 else (u_bar + (c**(1-eta)/(1-eta)))

@vectorize(nopython=True)
def du(c,eta):
    "Derivative of log utility function"
    return 1e10 if c < 1e-10 else c**(-eta)

@vectorize(nopython=True)
def duinv(u,eta):
    "Inverse of the derivative of the CRRA utility function"
    return u**(-1.0/eta )

@vectorize(nopython=True)
def f(X):
    "production function"
    return X

@vectorize(nopython=True)
def df(X):
    "Derivative of production function"
    return 1

@vectorize(nopython=True)
def discount_factor(beta, a_bar, c, w):
    "risk adjusted discount factor"
    return beta*a_bar*w/(c + a_bar*w)

@vectorize(nopython=True)
def next_x(X, N, g, c, w):
    # next period's CPR given c and w
    return (X - N*(c + w))*(1+g)

@vectorize(nopython=True)
def vector_dampen(vec1, vec2, dampen=1):
    return dampen*vec1 + (1-dampen)*vec2

This code block contains other functions that are useful for computing the policy using the envelope condition (to be discussed later), simulating, and computing Euler errors.

In [2964]:
@jit(nopython=True)
def env_cond_kp(temp, params, degree, v_coeffs, Xt):
    # Unpack parameters
    N, beta, g, a_bar, eta, u_bar = param_unpack(params)

    # Compute derivative of VF wrt k
    _complete_poly_der_impl_vec(np.array([Xt, zt]), degree, 0, temp)

    c = duinv(np.dot(temp, v_coeffs) / (1/N))

    return expendables_t(Xt) - c


@jit(nopython=True)
def jit_simulate_ncgm(params, degree, v_coeffs, T, nburn, shocks):
    "Simulates economy using envelope condition as policy rule"
    # Unpack parameters
    N, beta, g, a_bar, eta, u_bar = param_unpack(params)

    # Allocate space for output
    xsim = np.empty(T+nburn)
    zsim = np.empty(T+nburn)
    xsim[0], zsim[0] = 1.0, 1.0 # may need to modify initial values

    # Allocate space for temporary vector to fill with complete polynomials
    temp = np.empty(n_complete(2, degree))

    # Simulate
    for t in range(1, T+nburn):
        # Evaluate policy for today given yesterdays state
        xp = env_cond_kp(temp, params, degree, v_coeffs, xsim[t-1], zsim[t-1])
        
        # Draw new z and update k using policy from above
        zsim[t] = shocks[t]
        xsim[t] = kp

    return xsim[nburn:], zsim[nburn:]


@jit(nopython=True)
def jit_ee(params, degree, v_coeffs, nodes, weights, xs, zs):
    # Unpack parameters
    N, beta, g, a_bar, eta, u_bar = param_unpack(params)

    # Allocate space for temporary vector to fill with complete polynomials
    temp = np.empty(n_complete(2, degree))
    T = xs.size
    Qn = weights.size

    # Allocate space to store euler errors
    ee = np.empty(T)

    # Iterate over all ks and zs
    for t in range(T):
        # Current states
        x, z = xs[t], zs[t]

        # Compute decision for kp and implied c
        x1 = env_cond_kp(temp, params, degree, v_coeffs, x, z)
        c = expendables_t(k, z, A, alpha, delta) - k1

        # Compute euler error for period t
        lhs = du(c, gamma)
        rhs = 0.0
        for i in range(Qn):
            # Get productivity tomorrow
            z1 = z**rho * np.exp(nodes[i])

            # Compute decision for kpp and implied c
            k2 = env_cond_kp(temp, params, degree, v_coeffs, k1, z1)
            c1 = expendables_t(k1, z1, A, alpha, delta) - k2
            rhs = rhs + weights[i]*du(c1, gamma)*(1-delta+df(k1, z1, A, alpha))

        ee[t] = np.abs(1.0 - beta*rhs/lhs)

    return ee


We also now define a class that contains

1. Parameters of the growth model
2. Grids used for approximating the solution
3. Nodes and weights used to approximate integration

This again helps us maintain all of the relevant information in one place and simplifies passing it to other functions.

In [2965]:
class Model(object):
    """
    The stochastic Neoclassical Growth model contains
    parameters which include

    * alpha: Capital share in output
    * beta: discount factor
    * delta: depreciation rate
    * gamma: risk aversion
    * rho: persistence of the log productivity level
    * sigma: standard deviation of shocks to log productivity
    """
    def __init__(self, N=1, beta=0.9, g_x=0.005, g_a=0.015, a_bar=1000, 
                 eta=1, u_bar=0, 
                 xmin=10, xmax=280, nx=25):
        # calculate g from g_a and g_x 
        g = g_a + g_x + g_a*g_x

        # Household parameters
        self.beta, self.eta, self.u_bar, = beta, eta, u_bar

        # Firm/technology parameters
        self.N, self.g, self.a_bar = N, g, a_bar

        # Create t grids
        self.xgrid = np.linspace(xmin, xmax, nx)
        self.grid = np.column_stack((self.xgrid, np.log(self.xgrid)))
        
        self.ns = nx
        
    def _unpack_params(self):
        out = (self.N, self.beta, self.g, 
               self.a_bar, self.eta, self.u_bar)
        return out

    def _unpack_grids(self):
        out = (self.xgrid, self.grid)
        return out


## Solution Methods

In this notebook, we describe the following solution methods:

* Conventional Value Function Iteration
* Envelope Condition Value Function Iteration
* Envelope Condition Derivative Value Function Iteration
* Endogenous Grid Value Function Iteration
* Conventional Policy Function Iteration
* Envelope Condition Policy Function Iteration
* Euler Equation Method

Each of these solution methods will have a very similar structure that follows a few basic steps:

1. Guess a function (either value function or policy function).
2. Using this function, update our guess of both the value and policy functions.
3. Check whether the function we guessed and what it was updated to are similar enough. If so, proceed. If not, return to step 2 using the updated functions.
4. Output the policy and value functions.

In order to reduce the amount of repeated code and keep the exposition as clean as possible (the notebook is plenty long as is...), we will define a class for each solution method that inherit various properties from a general solution parent class. The parent class will contain methods which apply to all of the other solution methods such as a general solve method, computing expectations, simulating, etc... This implementation may feel strange at first if one isn't familiar with object oriented programming, but these concepts can be powerful when properly used.

In [2966]:
class GeneralSolution:
    """
    This is a general solution method. We define this, so that we can
    sub-class it and define specific update methods for each particular
    solution method
    """
    def __init__(self, ncgm, degree, prev_sol=None):
        # Save model and approximation degree
        self.ncgm, self.degree = ncgm, degree

        # Unpack some info from ncgm
        N, beta, g, a_bar, eta, u_bar = self._unpack_params()
        grid = self.ncgm.grid
        X = grid[:,0]

        # Use parameter values from model to create a namedtuple with
        # parameters saved inside
        self.params = Params(N, beta, g, a_bar, eta, u_bar)

        self.Phi = complete_polynomial(grid.T, degree).T
        self.dPhi = complete_polynomial_der(grid.T, degree, 0).T

        # Update to fill initial value and policy matrices
        # If we give it another solution type then use it to
        # generate values and policies
        if issubclass(type(prev_sol), GeneralSolution):
            oldPhi = complete_polynomial(ncgm.grid.T, prev_sol.degree).T
            self.CP = oldPhi @ prev_sol.c_coeffs
            self.WP = oldPhi @ prev_sol.w_coeffs
            self.VF = oldPhi @ prev_sol.v_coeffs
        # If we give it a tuple then assume it is (policy, value) pair
        elif type(prev_sol) is tuple:
            self.CP = prev_sol[0]
            self.WP = prev_sol[1]
            self.VF = prev_sol[2]
        # Otherwise guess a constant value function and a policy
        # of roughly steady state
        else:
            # guess VF
            self.VF = X # np.ones(ncgm.ns)
            
            # make a guess
            self.CP = X/(4*N)
            self.WP = X/(4*N)


        # Coefficients based on guesses
        self.c_coeffs = la.lstsq(self.Phi, self.CP)[0]
        self.w_coeffs = la.lstsq(self.Phi, self.WP)[0]
        self.v_coeffs = la.lstsq(self.Phi, self.VF)[0]

    def _unpack_params(self):
        return self.ncgm._unpack_params()

    def build_VF(self):
        """
        Using the current coefficients, this builds the value function
        for all states
        """
        VF = self.Phi @ self.v_coeffs

        return VF

    def build_CP(self):
        """
        Using the current coefficients, this builds the policy function
        for all states
        """
        CP = self.Phi @ self.c_coeffs

        return WP
    
    def build_WP(self):
        """
        Using the current coefficients, this builds the policy function
        for all states
        """
        WP = self.Phi @ self.w_coeffs

        return WP

    def update_v(self, new_v_coeffs, dampen=0.9):
        """
        Updates the coefficients for the value function
        """
        self.v_coeffs = (1-dampen)*self.v_coeffs + dampen*new_v_coeffs

        return None

    def update_c(self, new_c_coeffs, dampen=0.9):
        """
        Updates the coefficients for the policy function
        """
        self.c_coeffs = (1-dampen)*self.c_coeffs + dampen*new_c_coeffs

        return None
    
    def update_w(self, new_w_coeffs, dampen=0.9):
        """
        Updates the coefficients for the policy function
        """
        self.w_coeffs = (1-dampen)*self.w_coeffs + dampen*new_w_coeffs

        return None

    def update(self):
        """
        Given the current state of everything in solution, update the
        value and policy coefficients
        """
        emsg = "The update method is implemented in solution specific classes"
        emsg += "\nand cannot be called from `GeneralSolution`"
        raise ValueError(emsg)

    def compute_coefficients(self, cp, wp, VF):
        """
        Given a policy and value return corresponding coefficients
        """
        new_c_coeffs = la.lstsq(self.Phi, cp)[0]
        new_w_coeffs = la.lstsq(self.Phi, wp)[0]
        new_v_coeffs = la.lstsq(self.Phi, VF)[0]

        return new_c_coeffs, new_w_coeffs, new_v_coeffs

    def compute_EV_scalar(self, Xn):
        phi = complete_polynomial(np.vstack([Xn]), self.degree).T
        val = (phi@self.v_coeffs)[0]
        
        return val

    def compute_dEV_scalar(self, Xn):
        dphi = complete_polynomial_der(np.vstack([Xn]), self.degree,0).T
        val = (dphi@self.v_coeffs)[0]
        
        return val 

    def compute_EV(self, Xn=None):
        """
        Compute the expected value
        """
        # Unpack parameters
        N, beta, g, a_bar, eta, u_bar = self._unpack_params()

        # Use policy to compute cp and wp
        if Xn is None:
            cp = self.Phi @ self.c_coeffs
            wp = self.Phi @ self.w_coeffs
            X = self.ncgm.grid[:,0]
            Xn = next_x(X, N, g, cp, wp)
        
        Phi = complete_polynomial(np.vstack([Xn]), self.degree).T
        
        EV = Phi @ self.v_coeffs
        
        return EV


    def compute_dEV(self, Xn=None):
        """
        Compute the derivative of the expected value
        """
        # Unpack parameters
        N, beta, g, a_bar, eta, u_bar = self._unpack_params()

        # Use policy to compute cp and wp
        if Xn is None:
            cp = self.Phi @ self.c_coeffs
            wp = self.Phi @ self.w_coeffs
            X = self.ncgm.grid[:,0]
            Xn = next_x(X, N, g, cp, wp)
        
        dPhi = complete_polynomial_der(np.vstack([Xn]), self.degree).T
        
        dEV = dPhi @ self.v_coeffs
        return dEV

    def envelope_policy(self):
        """
        Applies the envelope condition to compute the policy for
        k_{t+1} at every point on the grid
        """
        # Unpack parameters
        A, alpha, beta, delta, gamma, rho, sigma = self._unpack_params()
        grid = self.ncgm.grid
        k, z = grid[:, 0], grid[:, 1]

        dV = self.dPhi@self.v_coeffs

        # Compute the consumption
        temp = dV / (1 - delta + df(k, z, A, alpha))
        c = duinv(temp, gamma)

        return expendables_t(k, z, A, alpha, delta) - c

    def compute_distance(self, cp, wp, VF):
        """
        Computes average distance between policy functions
        """
        return np.max([
            np.max(np.abs(1.0 - (cp+1e-12)/(self.CP+1e-12))), 
            np.max(np.abs(1.0 - (wp+1e-12)/(self.WP+1e-12)))
        ])
    

    def solve(self, tol=1e-3, maxiter=2500, verbose=False, nskipprint=25):
        # Iterate until convergence
        dist, it = 10.0, 0
        while (dist>tol) and (it<maxiter):
            # Run solution specific update code
            cp, wp, VF = self.update()

            # Compute new policy and value coeffs
            new_c_coeffs, new_w_coeffs, new_v_coeffs = self.compute_coefficients(cp, wp, VF)

            # Update distance and iterations
            dist = self.compute_distance(cp, wp, VF)
            self.CP, self.WP, self.VF = cp, wp, VF
            it += 1
            if verbose and it%nskipprint == 0:
                print(it, dist)

            # Update all coefficients
            self.update_c(new_c_coeffs)
            self.update_w(new_w_coeffs)
            self.update_v(new_v_coeffs)

        # After finishing iteration, iterate to convergence using policy
        if not isinstance(self, IterateOnPolicy):
            sol_iop = IterateOnPolicy(self.ncgm, self.degree, self)
            cp, wp, VF = sol_iop.solve(tol=1e-3)

            # Save final versions of everything
            self.CP, self.WP, self.VF = cp, wp, VF
            new_c_coeffs, new_w_coeffs, new_v_coeffs = sol_iop.compute_coefficients(cp, wp, VF)
            self.update_c(new_c_coeffs)
            self.update_w(new_w_coeffs)
            self.update_v(new_v_coeffs)

        return self.CP, self.WP, self.VF

    def simulate(self, T=10000, nburn=200, shocks=None, seed=42):
        """
        Simulates the neoclassical growth model with policy function
        given by self.KP. Simulates for `T` periods and discarsd first
        nburn `observations`
        """
        if shocks is None:
            np.random.seed(seed)
            shocks = np.random.exponential(self.z_bar, T+nburn)

        return jit_simulate_ncgm(self.params, self.degree, self.v_coeffs,
                                 T, nburn, shocks)

    def ee_residuals(self, ksim=None, zsim=None, Qn=10, seed=42):
        """
        Computes the Euler Equation residuals of a simulated capital and
        productivity levels (ksim and zsim) and uses Qn nodes for computing
        the expectation
        """
        if (ksim is None) or (zsim is None):
            ksim, zsim = self.simulate(T=10000, nburn=200, seed=seed)

        nodes, weights = qe.quad.qnwnorm(Qn, 0.0, self.ncgm.sigma**2)
        ee = jit_ee(self.params, self.degree, self.v_coeffs,
                    nodes, weights, ksim, zsim)

        return np.log10(np.mean(ee)), np.log10(np.max(ee))

### Iterating to Convergence (given policy)

This isn't one of the methods described above, but it is used as an element of a few of our methods (and also as a way to get a first guess at the value function). This method takes an initial policy function, $\bar{k}(k_t, z_t)$, as given, and then, without changing the policy, iterates until the value function has converged.

Thus the "update section" of the algorithm in this instance would be:

* Leave policy function unchanged
* At each point of grid, $(k_t, z_t)$, compute $\hat{V}(k_t, z_t) = u(c(\bar{k}(k_t, z_t))) + \beta E \left[ V(\bar{k}(k_t, z_t), z_{t+1}) \right]$

We override two of the methods from `GeneralSolution`

* `compute_distance` because when we are iterating to convergence on the value function we want to check distnace using value function rather than policy function
* `compute_coefficients` because we don't need to update the policy functions coefficients.

The `update` method just repeatedly applies a particular policy function to update the value function.

In [2967]:
class IterateOnPolicy(GeneralSolution):
    """
    Subclass of the general solution method. The update method for this
    class simply computes the fixed point of the value function given
    a specific policy
    """
    def compute_distance(self, cp, wp, VF):
        """
        Computes distance between policy functions. When we are
        iterating on a specific policy, we would like to compute
        distances by the difference between VFs
        """
        return np.max(np.abs(1.0 - (VF + 1e-12)/(self.VF + 1e-12)))

    def compute_coefficients(self, cp, wp, VF):
        """
        Given a policy and value return corresponding coefficients.
        When we are iterating on a specific policy, we don't want to
        update the policy coefficients.
        """
        new_v_coeffs = la.lstsq(self.Phi, VF)[0]

        return self.c_coeffs, self.w_coeffs, new_v_coeffs

    def update(self):
        # Unpack parameters
        N, beta, g, a_bar, eta, u_bar = self._unpack_params()
        X = self.ncgm.grid[:,0]
        
        Xn = next_x(X, N, g, self.CP, self.WP)

        # Update the value function
        VF = u(self.CP, eta, u_bar) + beta*np.maximum(self.compute_EV(Xn), np.zeros(self.ncgm.ns))

        return self.CP, self.WP, VF

### Conventional Value Function Iteration

This is one of the first solution methods for macroeconomics a graduate student in economics typically learns.

In this solution method, one takes as given a value function, $V(k_t, z_t)$, and then solves for the optimal policy given the value function.

The update section takes the form:

* For each point, $(k_t, z_t)$, numerically solve for $c^*(k_t, z_t)$ to satisfy the first order condition $u'(c^*) = \beta E \left[ V_1((1 - \delta) k_t + z_t f(k_t) - c^*, z_{t+1}) \right]$
* Define $k^*(k_t, z_t) = (1 - \delta) k_t + z_t f(k_t) - c^*(k_t, z_t)$
* Update value function according to $\hat{V}(k_t, z_t) = u(c^*(k_t, z_t)) + \beta E \left[ V(k^*(k_t, z_t), z_{t+1}) \right]$


* For each point, $(X_t, z_t)$, numerically solve for $c^*(X_t, z_t)$ to satisfy the first order condition $u'(\tilde{c}^\ast) = \beta E \left[ V_1((1 - \delta) k_t + z_t f(k_t) - c^*, z_{t+1}) \right]$
* Define $k^*(k_t, z_t) = (1 - \delta) k_t + z_t f(k_t) - c^*(k_t, z_t)$
* Update value function according to $\hat{V}(k_t, z_t) = u(c^*(k_t, z_t)) + \beta E \left[ V(k^*(k_t, z_t), z_{t+1}) \right]$




In [2968]:
class VFI(GeneralSolution):
    """
    Updates the coefficients and value functions using the VFI
    method
    """
    def update(self):
        """
        Updates the coefficients and value functions using the VFI_ECM
        method
        """
        # Unpack parameters
        N, beta, g, a_bar, eta, u_bar = self._unpack_params()
        grid = self.ncgm.grid
        n_state = grid.shape[0]

        # Get the policy and update it
        cp = np.empty(n_state)
        wp = np.empty(n_state)
        VF = np.empty(n_state)
        for i_s in range(n_state):
            # Pull out current vals
            X = grid[i_s, :][0]
            
            # objective function to minimize 
            def _f(CW):
                c = CW[0]
                w = CW[1]
                
                Xn = next_x(X, N, g, c, w)
                EV = self.compute_EV_scalar(Xn)
                dEV = self.compute_dEV_scalar(Xn)
                
                return (
                    du(c, eta) - (beta*a_bar*w/(c + a_bar*w))*(EV/(c + a_bar*w) + N*(1+g)*dEV)
                )**2 + (
                    (c/(c + a_bar*w))*EV - w*N*(1+g)*dEV
                )**2
                 
            
            A=[[1,1],[1,0],[0,1]]
            lbnd=0
            upbnd=X/N
            x0 = [X/(4*N),X/(4*N)]
            
            cons = opt.LinearConstraint(A, lbnd, upbnd)
            
            # minimize objective
            [_cp, _wp] = opt.minimize(_f, x0,constraints=cons, method='COBYLA').x
            
            Xn = next_x(X, N, g, _cp, _wp)
            EV = self.compute_EV_scalar(Xn)
            
            VF_corner = u(X/N, eta, u_bar)
            VF_interior = u(_cp, eta, u_bar)+beta*EV
            
            if VF_corner > VF_interior:
                cp[i_s] = X/N
                wp[i_s] = 0
                VF[i_s] = VF_corner
            else:                
                cp[i_s] = _cp
                wp[i_s] = _wp
                VF[i_s] = VF_interior

        return cp, wp, VF


In [2969]:
class find_thresh(GeneralSolution):
    """
    Updates the coefficients and value functions using the VFI
    method
    """
    def get_threshold(self):
        """
        Updates the coefficients and value functions using the VFI_ECM
        method
        """
        # Unpack parameters
        N, beta, g, a_bar, eta, u_bar = self._unpack_params()
        grid = self.ncgm.grid
        n_state = grid.shape[0]

        # Get the policy and update it
        cp = np.empty(n_state)
        wp = np.empty(n_state)
        VF = np.empty(n_state)
        for i_s in range(n_state):
            # Pull out current vals
            X = grid[i_s, :][0]
            
            # objective function to minimize 
            def _f(CW):
                c = CW[0]
                w = CW[1]
                
                Xn = next_x(X, N, g, c, w)
                
                return (
                    du(c, eta) - beta*((a_bar*w)/(c + a_bar*w))*(
                        u(Xn/N, eta, u_bar)/(c + a_bar*w)
                        + (1 + g)*du(Xn/N, eta)
                    )
                )**2 + (
                    (c/(c + a_bar*w))*u(Xn/N, eta, u_bar)
                    - w*(1 + g)*du(Xn/N, eta)
                )**2

            
            A=[[1,1],[1,0],[0,1]]
            lbnd=1e-6
            upbnd=X/N
            x0 = [0.01,0.01]
            
            cons = opt.LinearConstraint(A, lbnd, upbnd)
            
            # minimize objective
            [_cp, _wp] = opt.minimize(_f, x0, constraints=cons).x
            
            Xn = next_x(X, N, g, _cp, _wp)
            VF_interior = u(_cp, eta, u_bar) + beta*(a_bar*_wp/(_cp + a_bar*_wp))*u(Xn/N, eta, u_bar)
            
            VF_corner = u(X/N, eta, u_bar)
            
            if VF_corner > VF_interior:
                cp[i_s] = X/N
                wp[i_s] = 0
                VF[i_s] = VF_corner
            else:                
                cp[i_s] = _cp
                wp[i_s] = _wp
                VF[i_s] = VF_interior

        return cp, wp, VF

    


## A Horse Race

We can now run a horse race to compare the methods in terms of both accuracy and speed.

In [2970]:
ncgm = Model()

# First guess
vp = IterateOnPolicy(ncgm, 2)
vp.solve(tol=1e-9)

np.random.seed(61089)


print('vp done')
# degree 2 VFI:
d=2
new_sol = vp
new_sol = VFI(ncgm, d, new_sol)
ts = time.time()
new_sol.solve(tol=1e-2, verbose=True, nskipprint=25)
time_took = time.time() - ts

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 6 is different from 3)

In [ ]:
X = new_sol.ncgm.xgrid
X

In [ ]:
VF = new_sol.VF
VF

In [ ]:
new_sol.build_VF()

In [ ]:
CP = new_sol.CP
CP

In [ ]:
new_sol.build_CP()

In [ ]:
WP = new_sol.WP
WP

In [ ]:
new_sol.build_WP()

In [ ]:
CP/WP

In [ ]:
xnew = np.arange(0,1e3,1)
xnew_grid = np.reshape(xnew, [xnew.shape[0], 1]).T
Phi = complete_polynomial(xnew_grid, new_sol.degree).T
vf_new = Phi @ new_sol.v_coeffs

plt.plot(X, VF, 'o', xnew, vf_new)

In [ ]:
cp_new = Phi @ new_sol.c_coeffs

plt.plot(X, CP, 'o', xnew, cp_new)

In [ ]:
wp_new = Phi @ new_sol.w_coeffs

plt.plot(X, WP, 'o', xnew, wp_new)